# Generation of Mapping Model (MRRG)

## Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import pillars.core._
import pillars.hardware.{TopModule}
import pillars.archlib._
import pillars.core.{BlockTrait, OpEnum}
import pillars.core.OpEnum.OpEnum
import pillars.mapping.thirdParty._

## Create a simple block

In [ ]:

class SimpleBlock(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> term("out_0"))
}

## Get the mapping model: module routing resource graph (MRRG)
Box node: processing node for performing operators.

Circle node: routing node for routing values.

In [ ]:
val inputPort = 2
val outputPort = 1
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val block = new SimpleBlock("Block0")
arch.addBlock(block)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> block / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(block / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

val II = 1
val MRRG = arch.getMRRG(II)
var VizGraph = MRRG.GenGraph("Simple")
VizGraph.save("MRRG_simple", ".")
VizGraph.render(fileName = "MRRG_simple", directory = ".", format = "jpg")
viewDFG("./MRRG_simple.jpg")

## Get MRRG when initiation interval (II) = 2
To simplify the simulation flow, we set the input port can only transmit data at reconfiguration cycle 0.

In [ ]:
val II = 2
val MRRG = arch.getMRRG(II)
var VizGraph = MRRG.GenGraph("Simple")
VizGraph.save("MRRG_simple_II2", ".")
VizGraph.render(fileName = "MRRG_simple_II2", directory = ".", format = "jpg")
viewDFG("./MRRG_simple_II2.jpg")

## Create a block with register files

In [ ]:
class SimpleBlockWithReg(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)

    // A register file with 2 registers 
    val rf0 = new ElementRF("rf0", List(1, 1, 1, 32))
    //port sequnces outs: 0: out_0
    //port sequnces inputs: 0: input_0
    rf0.addOutPorts(Array("out_0"))
    rf0.addInPorts(Array("input_0"))
    addElement(rf0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> rf0 / "input_0")
    addConnect(rf0 / "out_0" -> term("out_0"))
}


## Get MRRG 
Due to the bug of Graphviz, the name of MRRG nodes are changed in examples of this tutorial.

In [ ]:
val inputPort = 2
val outputPort = 1
val dataWidth = 32

//Initialize the top block.
val arch = new ArchitectureHierarchy()
arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

val block = new SimpleBlockWithReg("Block0")
arch.addBlock(block)

(0 until inputPort).foreach(i =>
    arch.addConnect(arch.term(s"input_$i") -> block / s"input_$i"))
(0 until outputPort).foreach(i =>
    arch.addConnect(block / s"out_$i" -> arch.term(s"out_$i")))
arch.init()

val II = 1
val MRRG = arch.getMRRG(II)
var VizGraph = MRRG.GenGraph("Simple_Reg")
VizGraph.save("MRRG_simple_Reg", ".")
VizGraph.render(fileName = "MRRG_simple_Reg", directory = ".", format = "jpg")
viewDFG("./MRRG_simple_Reg.jpg")

## Get MRRG when initiation interval (II) = 2

In [ ]:
val II = 2
val MRRG = arch.getMRRG(II)
var VizGraph = MRRG.GenGraph("Simple_II2_Reg")
VizGraph.save("MRRG_simple_II2_Reg", ".")
VizGraph.render(fileName = "MRRG_simple_II2_Reg", directory = ".", format = "jpg")
viewDFG("./MRRG_simple_II2_Reg.jpg")

## Use block library to create a simple CGRA and get MRRG

In [ ]:
    val rowNum = 2
    val colNum = 2
    val inputPort = 1
    val outputPort = 1
    val dataWidth = 32

    //Initialize the top block.
    val arch = new ArchitectureHierarchy()
    arch.addInPorts((0 until inputPort).map(i => s"input_$i").toArray)
    arch.addOutPorts((0 until outputPort).map(i => s"out_$i").toArray)

    val tile = new TileLSUBlock("tile_0", colNum, rowNum, inputPort, outputPort,
      useMuxBypass = false, complex = true, isToroid = false, useCounter = true, dataWidth = dataWidth)
    arch.addBlock(tile)

    (0 until inputPort).foreach(i =>
      arch.addConnect(arch.term(s"input_$i") -> tile / s"input_$i"))
    (0 until outputPort).foreach(i =>
      arch.addConnect(tile / s"out_$i" -> arch.term(s"out_$i")))
    arch.init()

    val II = 1
    val MRRG = arch.getMRRG(II)
    var VizGraph = MRRG.GenGraph("VizGraph")
    VizGraph.save("MRRG_origin", ".")
    VizGraph.render(fileName = "MRRG_origin", directory = ".", format = "jpg")
    viewDFG("./MRRG_origin.jpg")
      
   